### Import

In [9]:
import os
import numpy as np
from tqdm import tqdm
from timm import create_model
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable
import torchvision
from torchvision.transforms import ToTensor, Compose, Resize, Normalize
import torchvision.models as models

### Data



In [10]:
transform = Compose([
    Resize((224, 224)),  # 調整到模型需求的輸入大小
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 常見正規化數值
])

train_ds = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/train', transform=transform)
valid_ds = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/vali_20k', transform=transform)
test_ds  = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/test_50k', transform=transform)

### Model

In [11]:
NUM_CLASSES = len(train_ds.class_to_idx)

alexnet = models.alexnet(pretrained=False, num_classes=NUM_CLASSES)

vgg16 = models.vgg16(pretrained=False)
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, NUM_CLASSES)
batch_norm = False  # 預設為 False
init_weights = True  # 預設為 True

####################### 需要額外處理 #######################
googlenet = models.googlenet(pretrained=False, aux_logits=True, transform_input=False)# 3. GoogleLeNet (Inception v1)
googlenet.fc = nn.Linear(googlenet.fc.in_features, NUM_CLASSES)
googlenet.aux1.fc2 = nn.Linear(googlenet.aux1.fc2.in_features, NUM_CLASSES)  # 修改輔助分類器
googlenet.aux2.fc2 = nn.Linear(googlenet.aux2.fc2.in_features, NUM_CLASSES)

resnet18 = models.resnet18(pretrained=False, zero_init_residual=False)
resnet18.fc = nn.Linear(resnet18.fc.in_features, NUM_CLASSES)

squeezenet = models.squeezenet1_0(pretrained=False, num_classes=NUM_CLASSES)
squeezenet.classifier[1] = nn.Conv2d(512, NUM_CLASSES, kernel_size=(1, 1), stride=(1, 1))

mobilenet_v2 = models.mobilenet_v2(pretrained=False, width_mult=1.0, round_nearest=8)
mobilenet_v2.classifier[1] = nn.Linear(mobilenet_v2.classifier[1].in_features, NUM_CLASSES)

efficientnet_b0 = models.efficientnet_b0(pretrained=False)
efficientnet_b0.classifier[1] = nn.Linear(efficientnet_b0.classifier[1].in_features, NUM_CLASSES)

swin_transformer = models.swin_t(pretrained=False, num_classes=NUM_CLASSES)
swin_transformer.head = nn.Linear(swin_transformer.head.in_features, NUM_CLASSES)
embed_dim = 96  # 預設
depths = [2, 2, 6, 2]  # 預設
num_heads = [3, 6, 12, 24]  # 預設
window_size = 7  # 預設
drop_path_rate = 0.1  # 預設
ape = False  # 預設
patch_norm = True  # 預設

xception = create_model('xception', pretrained=False, num_classes=NUM_CLASSES, drop_rate=0.2)

convnext_tiny = create_model('convnext_tiny', pretrained=False, num_classes=NUM_CLASSES, drop_path_rate=0.1)

mnasnet = models.mnasnet1_0(pretrained=False)
mnasnet.classifier[1] = nn.Linear(mnasnet.classifier[1].in_features, NUM_CLASSES)

### Training Parameters

In [12]:
# Model
model = alexnet

EPOCHS = 1
BATCH_SIZE = 256
LEARNING_RATE = 1e-3   
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 
loss_func = nn.CrossEntropyLoss()

num_workers = 4

### Setup

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    model.cuda()

### Train

In [14]:
print("Number of train samples: ", len(train_ds))
print("Number of test samples: ", len(test_ds))
print("Detected Classes are: ", train_ds.class_to_idx) 

train_loader = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
test_loader = data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)

# Train
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    # 使用 tqdm 包裹 train_loader
    train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training")
    for step, (x, y) in train_bar:
        x, y = x.to(device), y.to(device)
        b_x = Variable(x)
        b_y = Variable(y)

        # Forward
        output = model(b_x)
        loss = loss_func(output, b_y)

        # Train loss
        optimizer.zero_grad()           
        loss.backward()                 
        optimizer.step()

        # 更新 tqdm 的描述信息
        train_bar.set_postfix({"loss": loss.item()})


        torch.cuda.empty_cache()


        # # Test
        # if step % 50 == 0:
        #     test_x, test_y = next(iter(test_loader))
        #     test_x, test_y = test_x.to(device), test_y.to(device)
        #     test_output = model(test_x).argmax(1)
        #     accuracy = (test_output == test_y).sum().item() / BATCH_SIZE

        #     print(f"Step {step}: Test Accuracy: {accuracy:.2f}")

Number of train samples:  958941
Number of test samples:  49500
Detected Classes are:  {'dry_asphalt_severe': 0, 'dry_asphalt_slight': 1, 'dry_asphalt_smooth': 2, 'dry_concrete_severe': 3, 'dry_concrete_slight': 4, 'dry_concrete_smooth': 5, 'dry_gravel': 6, 'dry_mud': 7, 'fresh_snow': 8, 'ice': 9, 'melted_snow': 10, 'water_asphalt_severe': 11, 'water_asphalt_slight': 12, 'water_asphalt_smooth': 13, 'water_concrete_severe': 14, 'water_concrete_slight': 15, 'water_concrete_smooth': 16, 'water_gravel': 17, 'water_mud': 18, 'wet_asphalt_severe': 19, 'wet_asphalt_slight': 20, 'wet_asphalt_smooth': 21, 'wet_concrete_severe': 22, 'wet_concrete_slight': 23, 'wet_concrete_smooth': 24, 'wet_gravel': 25, 'wet_mud': 26}
Epoch 1/1


Training: 100%|██████████| 3746/3746 [28:50<00:00,  2.16it/s, loss=1.11] 


### Save Model


In [15]:
test_x, test_y = next(iter(test_loader))
test_x, test_y = test_x.to(device), test_y.to(device)
test_output = model(test_x).argmax(1)
accuracy = (test_output == test_y).sum().item() / BATCH_SIZE

In [16]:
model_name = type(model).__name__ 
file_name = f"{model_name}_{accuracy:.2f}_epoch{epoch+1}_batchSize{BATCH_SIZE}_lr{LEARNING_RATE:.0e}.pt"
save_path = os.path.join('../2_results/0_weight', file_name)
torch.save(model, save_path)